In [1]:
from dataset import *
from processing import *
from models import *

In [2]:
preprocess_midi_files('F:\\GitHub\\dataset\\midi_dataset', 'F:\\GitHub\\dataset\\np_dataset')

 [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 5.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 6 - 1st movement.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 7 - 2nd movement.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 9 - 2nd movement.mid] [F:\GitHub\dataset\midi_dataset\Beethoven - Symphony no. 9 - 4th movement.mid] [F:\GitHub\dataset\midi_dataset\Dukas - Sorcerer's Apprentice.mid] [F:\GitHub\dataset\midi_dataset\Hans_Zimmer_-_Pirates_Of_The_Caribbean_-_He's_A_Pirate.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Eine Kleine Nachtmusik.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Lacrimoza.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Marige of Figaro.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Queen of the Night.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Symphony no. 40.mid] [F:\GitHub\dataset\midi_dataset\Mozart - Symphony no. 41 - 3rd movement.mid] [F:\GitHub\dataset\midi_dataset\Paul Dukas - Sorcerer's Apprentice.m

C:\Users\Draco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [2]:
model = SimpleTransformer(VOCAB_SIZE, N_EMBD, N_HEAD, N_LAYER, FEEDFORWARD_DIM, BLOCK_SIZE, DROPOUT)
src = torch.randint(0, VOCAB_SIZE, (2, 100))  # Example input (batch_size=32, seq_len=10)
max_len = 150
gen_seq = model.generate(src, max_len)

In [3]:
gen_seq.shape

torch.Size([2, 150])

In [6]:
src.size(1)
src = src[:, -max_len:]
src[:, -150:].shape

torch.Size([2, 100])

In [ ]:
# src = torch.randint(0, VOCAB_SIZE, (2, 100))  # Example input (batch_size=32, seq_len=10)

trg = model(src)
last_pred = trg.argmax(2)[:, -1]
last_pred = last_pred.unsqueeze(1)  # Shape: [2, 1]
src = torch.cat((src, last_pred), dim=1)
print(src.shape)

RuntimeError: The size of tensor a (129) must match the size of tensor b (128) at non-singleton dimension 1

In [3]:
import torch.optim as optim

train_loader, test_loader = get_train_test_dataloaders('F:\\GitHub\\dataset\\np_dataset')

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0

    for input, target in train_loader:
        # Forward pass
        output = model(input, target)
        
        output = output.view(-1, output.size(-1))  # Flatten batch and seq_len dimensions
        target = target.view(-1)  # Flatten batch and seq_len dimensions
        
        # Compute loss
        loss = criterion(output, target)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss / len(train_loader):.4f}", end="\r")

print("Training complete.")

KeyboardInterrupt: 

In [5]:
for input, target in train_loader:
    # Forward pass
    output = model(input[0])
    # this is why

TypeError: TransformerNLP.forward() missing 1 required positional argument: 'tgt'